## Analyzing American Baby Name Trends

#### Create table and importingt the csv file

In [ ]:
DROP TABLE baby_names;

CREATE TABLE baby_names (
  year INT,
  first_name VARCHAR(64),
  sex VARCHAR(64),
  num INT
);
\copy baby_names FROM 'usa_baby_names.csv' DELIMITER ',' CSV HEADER;

#### Classic American names

In [ ]:
-- Select first names and the total babies with that first_name
SELECT first_name, sum(num)
From baby_names

-- Group by first_name and filter for those names that appear in all 101 years
GROUP BY first_name
HAVING count(year) = 101

-- Order by the total number of babies with that first_name, descending
ORDER BY sum(num) desc;

#### Timeless or trendy?

In [ ]:
-- Classify first names as 'Classic', 'Semi-classic', 'Semi-trendy', or 'Trendy'
-- Alias this column as popularity_type
SELECT first_name, SUM(num),
    CASE WHEN COUNT(year) > 80 THEN 'Classic'
        WHEN COUNT(year) > 50 THEN 'Semi-classic'
        WHEN COUNT(year) > 20 THEN 'Semi-trendy'
        ELSE 'Trendy' END AS popularity_type
FROM baby_names

-- Select first_name, the sum of babies who have ever had that name, and popularity_type
GROUP BY first_name

-- Order the results alphabetically by first_name
ORDER BY first_name;

#### Top-ranked female names since 1920

In [ ]:
-- RANK names by the sum of babies who have ever had that name (descending), aliasing as name_rank
-- Select name_rank, first_name, and the sum of babies who have ever had that name
SELECT
    RANK() OVER(ORDER BY SUM(num) DESC) AS name_rank,
    first_name, SUM(num)
FROM baby_names

-- Filter the data for results where sex equals 'F'
WHERE sex = 'F'
GROUP BY first_name

-- Limit to ten results
LIMIT 10;

#### Picking a baby name

In [ ]:
-- Select only the first_name column
SELECT first_name
FROM baby_names

-- Filter for results where sex is 'F', year is greater than 2015, and first_name ends in 'a'
WHERE sex = 'F' AND year > 2015
    AND first_name LIKE '%a'
    
-- Group by first_name and order by the total number of babies given that first_name
GROUP BY first_name
ORDER BY SUM(num) DESC;

#### The Olivia expansion

In [ ]:
-- Select year, first_name, num of Olivias in that year, and cumulative_olivias
-- Sum the cumulative babies who have been named Olivia up to that year; alias as cumulative_olivias
SELECT year, first_name, num,
    SUM(num) OVER (ORDER BY year) AS cumulative_olivias
FROM baby_names

-- Filter so that only data for the name Olivia is returned.
WHERE first_name = 'Olivia'

-- Order by year from the earliest year to most recent
ORDER BY year;


#### Many males with the same name

In [ ]:
-- Select year and maximum number of babies given any one male name in that year, aliased as max_num
SELECT year, MAX(num) AS max_num
FROM baby_names

-- Filter the data to include only results where sex equals 'M'
WHERE sex = 'M'
GROUP BY year;

#### Top male names over the years

In [ ]:
-- Select year, first_name given to the largest number of male babies, and num of babies given that name

SELECT b.year, b.first_name, b.num
FROM baby_names AS b

-- Join baby_names to the code in the last task as a subquery

INNER JOIN (
    SELECT year, MAX(num) as max_num
    FROM baby_names
    WHERE sex = 'M'
    GROUP BY year) AS subquery 
ON subquery.year = b.year 
    AND subquery.max_num = b.num
    
-- Order results by year descending
ORDER BY year DESC;

#### The most years at number one

In [ ]:
-- Select first_name and a count of years it was the top name in the last task; alias as count_top_name
-- Use the code from the previous task as a common table expression

WITH top_male_names AS (
    SELECT b.year, b.first_name, b.num
    FROM baby_names AS b
    INNER JOIN (
        SELECT year, MAX(num) num
        FROM baby_names
        WHERE sex = 'M'
        GROUP BY year) AS subquery 
    ON subquery.year = b.year 
        AND subquery.num = b.num
    ORDER BY YEAR DESC
    )
SELECT first_name, COUNT(first_name) as count_top_name
FROM top_male_names

-- Group by first_name and order by count_top_name descending
GROUP BY first_name
ORDER BY COUNT(first_name) DESC;